## Contents of the notebook

[Raghav Rastogi](https://www.kaggle.com/raghavrastogi75)

- [1 - Data Import](#1)
- [2 - Basic Analysis](#2)
- [3 - Exploratory Data Analysis](#3)
- [4 - Prediction](#4)
- [5 - Explanation](#5)
- [6 - Conclusion](#6)

### I have referred the book 'Hands-on Machine Learning with scikit-Learn, Keras, and Tensorflow' by Aurélien Géron and applied on this dataset. I highly recommend it if you are a beginner. If you have a question or feedback, do not hesitate to write and if you find this kernal helpful, please do not forget to **UPVOTE** 🙂

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Data import <a id="1"></a> <br>

In [ ]:
place = pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')

## Basic Analysis <a id="2"></a> <br>

Let's have a look at the Null values and the overall look at the data

In [ ]:
print(place.shape)
place.head(20)
print(place.isna().any())
#place = place.set_index('sl_no')
place.head()

## Exploratory Data Analysis <a id="3"></a> <br>

Looking at the numerical values of the data set

In [ ]:
print(place.describe())

Let's have a visualisation of different aspects of the dataset to get an idea of it and also get an intuition of the important parameters with respect to target variable 'Status'

In [ ]:
pv1 = place.pivot_table(index = 'gender', columns = 'status',values = 'ssc_p' )

x = np.arange(len(pv1.index))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize = (5,5))
rects1 = ax.bar(x - width/2, pv1['Not Placed'] , width, label='Not Placed')
rects2 = ax.bar(x + width/2, pv1['Placed'] , width, label='Placed')

ax.spines["top"].set_color("None")
ax.spines["right"].set_color("None")
ax.set_ylabel('Scores')
ax.set_title('Average Senior secondary percentage by Gender')
ax.set_xticks(x)
ax.set_ylim(0,100)
ax.set_xticklabels(pv1.index)
ax.legend()
plt.show()


Conclusion : We get a clear picture that senior secondary percentage is higher for students that got placed

In [ ]:
pv2 = place.pivot_table(index = 'gender', columns = 'status',values = 'etest_p' )

x = np.arange(len(pv2.index))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize = (5,5))
rects1 = ax.bar(x - width/2, pv2['Not Placed'] , width, label='Not Placed')
rects2 = ax.bar(x + width/2, pv2['Placed'] , width, label='Placed')

ax.spines["top"].set_color("None")
ax.spines["right"].set_color("None")
ax.set_ylabel('Scores')
ax.set_title('Average Employability test scores by Gender')
ax.set_xticks(x)
ax.set_ylim(0,100)
ax.set_xticklabels(pv1.index)
ax.legend()
plt.show()

Surprisingly not much clear difference in terms of placement.

In [ ]:
pv3 = place.pivot_table(index = 'gender', columns = 'status',values = 'mba_p' )

x = np.arange(len(pv3.index))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize = (5,5))
rects1 = ax.bar(x - width/2, pv3['Not Placed'] , width, label='Not Placed')
rects2 = ax.bar(x + width/2, pv3['Placed'] , width, label='Placed')

ax.spines["top"].set_color("None")
ax.spines["right"].set_color("None")
ax.set_ylabel('Scores')
ax.set_title('Average MBA percentage by Gender')
ax.set_xticks(x)
ax.set_ylim(0,100)
ax.set_xticklabels(pv1.index)
ax.legend()
plt.show()

Not much difference of scores here as well.

In [ ]:
pv4 = place.pivot_table(index = 'degree_t', columns = 'status', values = 'gender', aggfunc = 'count')
print(pv4)
x = np.arange(len(pv4.index))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize = (5,5))
rects1 = ax.bar(x - width/2, pv4['Not Placed'] , width, label='Not Placed')
rects2 = ax.bar(x + width/2, pv4['Placed'] , width, label='Placed')

ax.spines["top"].set_color("None")
ax.spines["right"].set_color("None")
ax.set_ylabel('Count')
ax.set_title('Count of placement status of students by degree')
ax.set_xticks(x)
ax.set_ylim(0,110)
ax.set_xticklabels(pv4.index)
ax.legend()
plt.show()

We see that students with Commerce and Management degree have more placements although the placements percentage is more for Science & technology

In [ ]:
pv5 = place.pivot_table(index = 'specialisation', columns = 'status', values = 'gender', aggfunc = 'count')
pv5

In terms of 'specialization' students got placed much more in 'Market and Finance'.

## Prediction <a id="4"></a> <br>

Importing all libraries and creating the test and train dataset. Also separating the numerical and the categorical attributes to apply different data processing techniques.


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier

place_new = place.drop('salary',axis = 1).reset_index()
place_new = place_new.replace(['Placed','Not Placed'],[1,0])
# print(place_new,'place_new')
place_cat = place_new[['gender','ssc_b','hsc_b','hsc_s','degree_t','workex','specialisation']]
place_num = place_new[['ssc_p','hsc_p','degree_p','etest_p','mba_p']]

X = place_new.drop(['status'],axis = 1).reset_index().drop(['sl_no'],axis = 1)
y = place_new.iloc[:,-1]


Using Pipeline and ColumnTransfer I have combined all the data processing and prediction algorithms for both numerical and categorical data. 

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(place_num)
cat_attribs = list(place_cat)
# print(num_attribs,"***num_Attribs***")
# print(cat_attribs,"***cat_attribs***")
num_transformer = Pipeline(steps = [('scaler', StandardScaler())])

cat_transformer = Pipeline(steps = [('onehot',OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers = [('num',num_transformer, num_attribs),
                                                 ('cat',cat_transformer,cat_attribs)])

param_grid = [{'n_estimators':[100,500],'max_features': [8, 10, 12]},
              {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}]

forest_clf = RandomForestClassifier()


clf = Pipeline(steps = [('preprocessor',preprocessor),('grid_search',GridSearchCV(forest_clf, param_grid, cv=5,
scoring='roc_auc',return_train_score=True))])

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)

for train_index, test_index in split.split(X,y):
    #print(train_index,(test_index))
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
#print(y_pred)


from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

print("model score: %.3f" % clf.score(X_test, y_test))


## Explanation:<a id="5"></a> <br> 
* I created 'num_attribs' and 'cat_attribs' and I created Pipeline for data preprocessing for both Numeric and Categorical data.
* Then I combined the two using Column transfer and created a combined variable 'preprocessor' which can be used directly in a pipeline later on.
* Created a 'param_grid' which is the combined list of parameters to be tested for grid search.
* Using Forest Classifier for prediction.
* Performing grid search cross validation with Forest Classifier with scoring as 'roc_auc' using pipeline and storing it as clf.
* Applying stratified shuffle split which will create a test set that is a complete representative of the data for example the gender ratio will remain constant for the test set as well.
* Finally fitting the data set on the classifier
* Creating y_pred to create a list of predicted variable
* Using confusion matrix to find the false negetives and false positives.
* Finally getting the roc_auc score.

In [ ]:
print(clf.named_steps['grid_search'].best_params_)

As we can see the best parameters for Grid Search are max_features = 8 and n_estimators = 500. I might try to tune it even finer in future iterations.

# Conclusion<a id="6"></a> <br>

This is one of my initial attempts on EDA and prediction.
Do let me know if there are any feedback/comments/suggestions :) 
I would to know how I can improve further and make even better predictions.